In [85]:
import requests
from bs4  import BeautifulSoup
import pandas as pd

In [86]:
# Loading the page
url = 'https://forecast.weather.gov/MapClick.php?x=276&y=148&site=lox&zmx=&zmy=&map_x=276&map_y=148#.YitDzujMLIU'

page = requests.get(url)

# Checking if the request was successful
print(page)


# Preparing the soup :)
soup = BeautifulSoup(page.content, 'html.parser')


<Response [200]>


In [87]:
#extract days of the week

week_day = []
for line in soup.find_all('div', class_ ='col-sm-2 forecast-label'):
    week_day.append(line.text)


# extract weather description
description = []
for line in soup.find_all('div', class_ ='col-sm-10 forecast-text'):
    description.append( line.text )



# extract temperature
# low temp




In [88]:
# date of the day
dates = pd.date_range(start= pd.to_datetime("today"), periods=7)


# account for day and night pattern (repeating the same date twice)
day_night = []
for i, j in zip(dates, dates):
    day_night.append(i.date())
    day_night.append(j.date())


# last day of the week in `week_day` appears only once
day_night = day_night[ :len(day_night) - 1]


In [89]:
# join scraped data into a dataframe

weather_data = pd.DataFrame({'day_night': day_night,
               'week_day': week_day,
               'weather_description': description})


weather_data


,day_night,week_day,weather_description
0,2022-03-11,Today,"Sunny, with a high near 74. Northeast wind 5 t..."
1,2022-03-11,Tonight,"Mostly clear, with a low around 45. Northwest ..."
2,2022-03-12,Saturday,"Sunny, with a high near 76. North northeast wi..."
3,2022-03-12,Saturday Night,"Partly cloudy, with a low around 49. West sout..."
4,2022-03-13,Sunday,"Mostly sunny, with a high near 71. East northe..."
5,2022-03-13,Sunday Night,"Partly cloudy, with a low around 49."
6,2022-03-14,Monday,"Sunny, with a high near 76."
7,2022-03-14,Monday Night,"Mostly clear, with a low around 53."
8,2022-03-15,Tuesday,"Mostly sunny, with a high near 75."
9,2022-03-15,Tuesday Night,"Partly cloudy, with a low around 52."
